##Importing libraries and packages

In [ ]:
!pip install rasterio
!pip install geotiff
!pip install imagecodecs-lite
!pip install imagecodecs

In [96]:
import os
import json
import math
import time
import urllib.request

import requests
from requests.auth import HTTPBasicAuth
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
import imageio
from osgeo import gdal
import skimage
import skimage.io as skio
from PIL import Image as pilimg
from skimage import exposure

# Image Preparation

In [123]:
# Input your key as a string here
PLANET_API_KEY = ''

In [98]:
def create_filter(coords, i = 0, j = 0):
    """
    TODO?: Modify filters
    """
    low_latitude = coords[0]
    high_latitude = coords[1]
    low_longitude = coords[2]
    high_longitude = coords[3]

    low_latitude += 0.06 * i
    high_latitude += 0.06 * i
    low_longitude += 0.06 * j
    high_longitude += 0.06 * j
    geojson_geometry = {
        "type": "Polygon",
        "coordinates": [
        [ 
            [low_latitude, low_longitude],
            [high_latitude, low_longitude],
            [high_latitude, high_longitude],
            [low_latitude, high_longitude],
            [low_latitude, low_longitude]
        ]
        ]
    }
    # get images that overlap with our AOI 
    geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geojson_geometry
    }

    # get images acquired within a date range
    date_range_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config": {
        "gte": start_time,
        "lte": end_time
    }
    }

    # only get images which have <50% cloud coverage
    cloud_cover_filter = {
    "type": "RangeFilter",
    "field_name": "cloud_cover",
    "config": {
        "lte": 0.5
    }
    }

    # combine our geo, date, cloud filters
    combined_filter = {
    "type": "AndFilter",
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
    }
    return combined_filter

In [121]:
images_already_fetched = []
def create_url(combined_filter, latitude, longitude):
    """
    TODO: Grab more than the first image ID
    TODO?: Check folder if image exists
    """

    item_type = "PSScene4Band"

    # API request object
    search_request = {
    "item_types": [item_type], 
    "filter": combined_filter
    }

    # fire off the POST request
    search_result = \
    requests.post(
        'https://api.planet.com/data/v1/quick-search',
        auth=HTTPBasicAuth(PLANET_API_KEY, ''),
        json=search_request)

    # print(json.dumps(search_result.json(), indent=1))
        
    image_ids = [feature['id'] for feature in search_result.json()['features']]
    print("Image IDs: ", image_ids)

    if not image_ids:
        print("No images found for Lat:", latitude, " Long:", longitude)
        return None

    # For our purposes, just grab the first image ID
    id0 = image_ids[0]

    # The same image id is returned for several different queries. If the image id
    # is already seen before then skip processing it further.
    if id0 in images_already_fetched:
        print("Skipping:" + id0 + " because already processed earlier")
        return None
    else:
        images_already_fetched.append(id0)
        id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)
    return id0_url



In [110]:
def download_image(url):
    """
    TODO: Come up with a filename convention for these pictures (i.e by image ID)
    """
    # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
    result = \
      requests.get(
        url,
        auth=HTTPBasicAuth(PLANET_API_KEY, '')
      )

    # List of asset types available for this particular satellite image
    print(result.json().keys())   

    # This is "inactive" if the "analytic" asset has not yet been activated; otherwise 'active'
    print(result.json()['analytic']['status'])

    # Parse out useful links
    links = result.json()[u"analytic_sr"]["_links"]
    self_link = links["_self"]
    activation_link = links["activate"]

    # Request activation of the 'analytic' asset:
    activate_result = \
      requests.get(
        activation_link,
        auth=HTTPBasicAuth(PLANET_API_KEY, '')
      )

    activation_status_result = \
      requests.get(
        self_link,
        auth=HTTPBasicAuth(PLANET_API_KEY, '')
      )
        
    print(activation_status_result.json()["status"])

    while(activation_status_result.json()["status"] != 'active'):
      time.sleep(60)
      activation_status_result = \
      requests.get(
        self_link,
        auth=HTTPBasicAuth(PLANET_API_KEY, '')
      )
      print(activation_status_result.json()["status"])



    # Image can be downloaded by making a GET with your Planet API key, from here:
    download_link = activation_status_result.json()["location"]
    print(download_link)

    image_file = download_link

    urllib.request.urlretrieve(image_file, filename="test.tif")

In [111]:
def tif_to_jpg(filename):
    # Convert the image from tif to jpeg
    image = skio.imread('test.tif')
    clamp_low = np.quantile(image, .0005)
    clamp_high = np.quantile(image, .9905)
    image = exposure.rescale_intensity(image, in_range=(clamp_low, clamp_high))

    clipped = skimage.img_as_ubyte(image) #converts an image to an unsigned byte format, with values between [0,255] 
    # print(clipped[clipped>0]) #returns indexes of clipped with the value that satisfies clipped>0 condition
    b = clipped[:,:, 0]
    g = clipped[:,:, 1]
    r = clipped[:,:, 2]
    n = clipped[:,:, 3]
    rgb = np.stack([r, g, b])
    rgb = np.moveaxis(rgb, 0, -1)
    plt.figure(figsize=(25, 13))
    skio.imsave(filename, rgb) #saves clipped from a geotif to a tif
    skio.imshow(filename) #displaying the file


In [112]:
import rasterio
import numpy as np
import imageio
def tif_to_jpg2(filename):
    """
    Currently doesn't work
    """
    image_file = "test.tif"  # Change to whatever the planet image is that we download!

    sat_data = rasterio.open(image_file)

    # (integer unsigned 16-bit, if planet.com is right)
    b, g, r, n = sat_data.read()

    # Approach that uses numpy to make an array and then imageio to write an image:
    image_array = np.array([r, g, b]).astype(np.uint8).T # Forces to uint8...
    imageio.imwrite(filename, image_array)

In [113]:
def get_coords(aoi):
    coords = aoi['features'][0]['geometry']['coordinates'][0]
    low_latitude = coords[0][0]
    high_latitude = coords[1][0]
    low_longitude = coords[0][1]
    high_longitude = coords[2][1]
    return (low_latitude, high_latitude, low_longitude, high_longitude)

# Query Parameters

Add AOI JSON as a dictionary below:

In [114]:
mazabuka_aoi = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              27.58255004882812,
              -16.22786079204794
            ],
            [
              28.208770751953125,
              -16.22786079204794
            ],
            [
              28.208770751953125,
              -15.634939470864143
            ],
            [
              27.58255004882812,
              -15.634939470864143
            ],
            [
              27.58255004882812,
              -16.22786079204794
            ]
          ]
        ]
      }
    }
  ]
}

In [115]:
mazabuka_coords = get_coords(mazabuka_aoi)

Select time frame:

In [116]:
# Start and end time stamp in ISO8601 format
start_time = "2020-08-15T00:00:00.000Z"
end_time = "2020-08-16T00:00:00.000Z"

In [117]:
def prepare_images(coords, start_time, end_time, i = 1, j = 1):
    """
    Create filter, create url, download image from url, convert tiff to jpg
    """
    for latitude in range(i):
        for longitude in range(j):
            combined_filter = create_filter(coords)
            id_url = create_url(combined_filter, latitude, longitude)
            if id_url != None:
                download_image(id_url)
                tif_to_jpg('lat_' + str(latitude) + '_long_' + str(longitude) + '.jpeg')

In [ ]:
prepare_images(mazabuka_coords, start_time, end_time)

# Image Processing

In [ ]:
skio.imshow('lat_0_long_0.jpeg')